In [ ]:
%load_ext sagemaker_studio_analytics_extension.magics
%sm_analytics emr connect --cluster-id  --auth-type None 
# Add EMR cluster ID after "--cluster-id". The id should start with j- and can be retrieved from the Cluster button on SageMaker Studio Notebook or from the EMR console

# Ensuring Consistency with ACID Transactions with Delta Lake (Loan Risk Data)

<img src="https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-logo-whitebackground.png" width=200/>

This is a companion notebook to provide a Delta Lake example against the Lending Club data.
* This notebook has been tested with *EMR 6.6.0, SparkMagic Kernel*

## The Data

The data used is public data from Lending Club. It includes all funded loans from 2012 to 2017. Each loan includes applicant information provided by the applicant as well as the current loan status (Current, Late, Fully Paid, etc.) and latest payment information. For a full view of the data please view the data dictionary available [here](https://resources.lendingclub.com/LCDataDictionary.xlsx).


![Loan_Data](https://preview.ibb.co/d3tQ4R/Screen_Shot_2018_02_02_at_11_21_51_PM.png)


The Following notebook was adapted from [this Databricks Notebook](https://databricks-prod-cloudfront.cloud.databricks.com/public/4027ec902e239c93eaaa8714f173bcfc/8681037038499168/2169405638094745/5617296914462207/latest.html).

## ![Delta Lake Tiny Logo](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) Delta Lake

Optimization Layer a top blob storage for Reliability (i.e. ACID compliance) and Low Latency of Streaming + Batch data pipelines.

## Import Data and create pre-Delta Lake Table
* This will create a lot of small Parquet files emulating the typical small file problem that occurs with streaming or highly transactional data

In [42]:
%%configure -f 

{
  "conf": {
    "spark.jars.packages": "io.delta:delta-core_2.12:2.0.0",
    "spark.sql.extensions": "io.delta.sql.DeltaSparkSessionExtension",
    "spark.sql.catalog.spark_catalog": "org.apache.spark.sql.delta.catalog.DeltaCatalog"
  }
}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
9,application_1683644616679_0010,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
2,application_1683644616679_0003,pyspark,idle,Link,Link,None,
4,application_1683644616679_0005,pyspark,idle,Link,Link,None,
5,application_1683644616679_0006,pyspark,idle,Link,Link,None,
7,application_1683644616679_0008,pyspark,idle,Link,Link,None,
8,application_1683644616679_0009,pyspark,idle,Link,Link,None,
9,application_1683644616679_0010,pyspark,idle,Link,Link,None,✔


In [51]:
from pyspark.sql import SparkSession
from delta.tables import *

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Download the data from [Databricks notebook](https://databricks-prod-cloudfront.cloud.databricks.com/public/4027ec902e239c93eaaa8714f173bcfc/8681037038499168/2169405638094745/5617296914462207/latest.html) and upload it to your S3 bucket. Copy the bucket path and paste it into the cell below.

In [52]:
# Configure location of loanstats_2012_2017.parquet
lspq_path = ""

# Read loanstats_2012_2017.parquet
data = spark.read.parquet(lspq_path)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [59]:
# Reduce the amount of data (to run on DBCE)
(loan_stats, loan_stats_rest) = data.randomSplit([0.01, 0.99], seed=123)


# Select only the columns needed
loan_stats = loan_stats.select("addr_state", "loan_status")

# Create loan by state
loan_by_state = loan_stats.groupBy("addr_state").count()

# Create table
loan_by_state.createOrReplaceTempView("loan_by_state")

# Display loans by state
loan_by_state.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-----+
|addr_state|count|
+----------+-----+
|        MN|  254|
|        DC|   47|
|        MD|  360|
|        DE|   44|
|        MO|  269|
+----------+-----+
only showing top 5 rows

## ![Delta Lake Tiny Logo](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) Easily Convert Parquet to Delta Lake format
With Delta Lake, you can easily transform your Parquet data into Delta Lake format.

In [29]:
# Configure Delta Lake Raw & Silver Path
DELTALAKE_RAW_PATH = "s3://"
DELTALAKE_SILVER_PATH = "s3://"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
%%sql -q
CREATE DATABASE IF NOT EXISTS vedjain_loans_db LOCATION 's3://'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [80]:
%%sql -q
DROP TABLE IF EXISTS vedjain_loans_db.loan_by_state_delta;

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
data.write.mode("overwrite").format("delta").partitionBy("addr_state").save(DELTALAKE_RAW_PATH)
loan_by_state.write.mode("overwrite").format("delta").partitionBy("addr_state").save(DELTALAKE_SILVER_PATH)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [35]:
%%sql -q

CREATE TABLE vedjain_loans_db.loan_delta
USING DELTA
LOCATION ''

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [46]:
%%sql -q

CREATE TABLE vedjain_loans_db.loan_by_state_delta
USING DELTA
LOCATION ''

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [55]:
%%sql 
SELECT * FROM vedjain_loans_db.loan_by_state_delta

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [56]:
%%sql 
DESCRIBE TABLE vedjain_loans_db.loan_by_state_delta

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [256]:
%%bash
# View .json files inside the _delta_log directory
aws s3 ls s3://

2023-04-27 19:17:28      16702 00000000000000000000.json
2023-04-27 19:46:14       1048 00000000000000000001.json
2023-04-27 19:52:58       2130 00000000000000000002.json
2023-04-27 19:56:28      20543 00000000000000000003.json
2023-04-27 19:58:14      39728 00000000000000000004.json
2023-04-27 19:58:58      47028 00000000000000000005.json
2023-04-27 19:59:09      69583 00000000000000000006.json
2023-04-27 20:03:25      92923 00000000000000000007.json
2023-04-27 20:04:05     135583 00000000000000000008.json
2023-04-27 20:04:17     200108 00000000000000000009.json


Review current loans within the `loan_by_state_delta` Delta Lake table

In [269]:
%%sql
select addr_state, sum(`count`) as loans from vedjain_loans_db.loan_by_state_delta group by addr_state

An error was encountered:
Invalid status code '404' from http://ip-10-0-20-55.us-east-2.compute.internal:8998/sessions/5 with error payload: {"msg":"Session '5' not found."}


Observe that the Iowa (middle state) has the largest number of loans due to the recent stream of data.  Note that the original `loan_by_state_delta` table is updated as we're reading `loan_by_state_readStream`.

In [270]:
### ![Delta Lake Logo Tiny](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) Full DML Support

**Note**: Full DML Support is a feature that will be coming soon to Delta Lake; the preview is currently available in Databricks.

Delta Lake supports standard DML including UPDATE, DELETE and MERGE INTO providing developers more controls to manage their big datasets.

An error was encountered:
Invalid status code '404' from http://ip-10-0-20-55.us-east-2.compute.internal:8998/sessions/5 with error payload: {"msg":"Session '5' not found."}


### ![Delta Lake Logo Tiny](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) Full DML Support

**Note**: Full DML Support is a feature that will be coming soon to Delta Lake; the preview is currently available in Databricks.

Delta Lake supports standard DML including UPDATE, DELETE and MERGE INTO providing developers more controls to manage their big datasets.

Let's start by creating a traditional Parquet table

In [260]:
# Load new DataFrame based on current Delta table

spark.sql("use vedjain_loans_db;")
lbs_df = spark.sql("select * from loan_by_state_delta")

# Save DataFrame to Parquet
lbs_df.write.mode("overwrite").parquet("/tmp/loan_by_state.parquet")

# Create new table on this parquet data
spark.sql("drop table if exists loan_by_state_pq")
spark.sql("create table loan_by_state_pq using parquet as select * from parquet.`/tmp/loan_by_state.parquet`")

# Review data
spark.sql("select * from loan_by_state_pq").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-----+------------------+
|addr_state|count|            amount|
+----------+-----+------------------+
|        NM|    2|24187.511136077177|
|        NM|    0| 4325.965858470662|
|        NM|    1|10086.698942103516|
|        NM|    9|   97599.657646052|
|        NM|   15|157243.11483596396|
|        NM|    1|18414.832038580484|
|        NM|    0|               0.0|
|        NM|    0|               0.0|
|        NM|    2|27057.016750416627|
|        NM|    0|               0.0|
|        NM|    0|               0.0|
|        NM|   11|118262.53117574497|
|        CA|    3| 39328.79659883139|
|        CA|    0|               0.0|
|        CA|    7| 73087.09786660112|
|        CA|   12|122057.63029860397|
|        CA|    1|12810.329732217462|
|        CA|   27|274657.10495015694|
|        CA|    0|               0.0|
|        CA|  207|2073483.2156239853|
+----------+-----+------------------+
only showing top 20 rows

None

### ![Delta Lake Logo Tiny](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) DELETE Support

The data was originally supposed to be assigned to `WA` state, so let's `DELETE` those values assigned to `IA`

In [145]:
%%sql
-- Attempting to run `DELETE` on the Parquet table
DELETE FROM loan_by_state_pq WHERE addr_state = 'IA'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
DELETE is only supported with v2 tables.
Traceback (most recent call last):
  File "/mnt1/yarn/usercache/livy/appcache/application_1682619508681_0004/container_1682619508681_0004_01_000001/pyspark.zip/pyspark/sql/session.py", line 723, in sql
    return DataFrame(self._jsparkSession.sql(sqlQuery), self._wrapped)
  File "/mnt1/yarn/usercache/livy/appcache/application_1682619508681_0004/container_1682619508681_0004_01_000001/py4j-0.10.9.2-src.zip/py4j/java_gateway.py", line 1310, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/mnt1/yarn/usercache/livy/appcache/application_1682619508681_0004/container_1682619508681_0004_01_000001/pyspark.zip/pyspark/sql/utils.py", line 117, in deco
    raise converted from None
pyspark.sql.utils.AnalysisException: DELETE is only supported with v2 tables.



**Note**: This command fails because the `DELETE` statements are not supported in Parquet, but are supported in Delta Lake.

In [146]:
%%sql
-- Running `DELETE` on the Delta Lake table
DELETE FROM loan_by_state_delta WHERE addr_state = 'IA'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [147]:
%%sql
-- Review current loans within the `loan_by_state_delta` Delta Lake table
select addr_state, sum(`count`) as loans from loan_by_state_delta group by addr_state

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

### ![Delta Lake Logo Tiny](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) UPDATE Support
The data was originally supposed to be assigned to `WA` state, so let's `UPDATE` those values

In [149]:
%%sql
-- Attempting to run `UPDATE` on the Parquet table
UPDATE loan_by_state_pq SET `count` = 2700 WHERE addr_state = 'WA'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
UPDATE destination only supports Delta sources.
Some(Relation vedjain_loans_db.loan_by_state_pq[addr_state#7740,count#7741L] parquet
)
Traceback (most recent call last):
  File "/mnt1/yarn/usercache/livy/appcache/application_1682619508681_0004/container_1682619508681_0004_01_000001/pyspark.zip/pyspark/sql/session.py", line 723, in sql
    return DataFrame(self._jsparkSession.sql(sqlQuery), self._wrapped)
  File "/mnt1/yarn/usercache/livy/appcache/application_1682619508681_0004/container_1682619508681_0004_01_000001/py4j-0.10.9.2-src.zip/py4j/java_gateway.py", line 1310, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/mnt1/yarn/usercache/livy/appcache/application_1682619508681_0004/container_1682619508681_0004_01_000001/pyspark.zip/pyspark/sql/utils.py", line 117, in deco
    raise converted from None
pyspark.sql.utils.AnalysisException: UPDATE destination only supports Delta sources.
Some(Relation vedjain_loans_db.loan_by_state

**Note**: This command fails because the `UPDATE` statements are not supported in Parquet, but are supported in Delta Lake.

In [150]:
%%sql
-- Running `UPDATE` on the Delta Lake table
UPDATE loan_by_state_delta SET `count` = 2700 WHERE addr_state = 'WA'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [151]:
%%sql
-- Review current loans within the `loan_by_state_delta` Delta Lake table
select addr_state, sum(`count`) as loans from loan_by_state_delta group by addr_state

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

### ![Delta Lake Logo Tiny](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) MERGE INTO Support

#### INSERT or UPDATE parquet: 7-step process

With a legacy data pipeline, to insert or update a table, you must:
1. Identify the new rows to be inserted
2. Identify the rows that will be replaced (i.e. updated)
3. Identify all of the rows that are not impacted by the insert or update
4. Create a new temp based on all three insert statements
5. Delete the original table (and all of those associated files)
6. "Rename" the temp table back to the original table name
7. Drop the temp table

![](https://pages.databricks.com/rs/094-YMS-629/images/merge-into-legacy.gif)


#### INSERT or UPDATE with Delta Lake

2-step process: 
1. Identify rows to insert or update
2. Use `MERGE`

In [153]:
# Let's create a simple table to merge
items = [('IA', 10), ('CA', 2500), ('OR', None)]
cols = ['addr_state', 'count']
merge_table = spark.createDataFrame(items, cols)
merge_table.createOrReplaceTempView("merge_table")
merge_table.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-----+
|addr_state|count|
+----------+-----+
|        IA|   10|
|        CA| 2500|
|        OR| null|
+----------+-----+

Instead of writing separate `INSERT` and `UPDATE` statements, we can use a `MERGE` statement.

In [154]:
%%sql
MERGE INTO loan_by_state_delta as d
USING merge_table as m
on d.addr_state = m.addr_state
WHEN MATCHED THEN 
  UPDATE SET *
WHEN NOT MATCHED 
  THEN INSERT *

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [155]:
%%sql
-- Review current loans within the `loan_by_state_delta` Delta Lake table
select addr_state, sum(`count`) as loans from loan_by_state_delta group by addr_state

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

## ![Delta Lake Logo Tiny](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) Schema Evolution
With the `mergeSchema` option, you can evolve your Delta Lake table schema

In [169]:
# Generate new loans with dollar amounts 
loans = spark.sql("select addr_state, cast(rand(10)*count as bigint) as count, cast(rand(10) * 10000 * count as double) as amount from loan_by_state_delta")
loans.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-----+------------------+
|addr_state|count|            amount|
+----------+-----+------------------+
|        NJ|    7| 71798.87979413386|
|        NJ|    6|64409.151664043675|
|        NJ|   90| 906820.3155244759|
|        NJ|   90| 900224.5476386878|
|        IL|    9| 90029.31403581101|
|        IL|   57| 571961.8728399859|
|        IL|  336| 3360734.282202703|
|        IL|  116|1163028.3796527511|
|        UT|   28|287813.02265224216|
|        UT|   13|131608.22770693692|
|        UT|    8| 80625.03712025726|
|        UT|    4| 49748.60737241262|
|        CA|   17|171473.88201575977|
|        CA|   60|  609997.860287825|
|        CA|    1| 19059.77149526836|
|        CA|  554| 5546375.977826152|
|        KS|    7| 77610.02768015937|
|        KS|   18| 180732.8637036786|
|        KS|    6| 66165.93838718785|
|        KS|    0|               0.0|
+----------+-----+------------------+
only showing top 20 rows

In [171]:
# Let's write this data out to our Delta table
loans.write.format("delta").mode("append").save(DELTALAKE_SILVER_PATH)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

**Note**: This command fails because the schema of our new data does not match the schema of our original data

In [172]:
# Add the mergeSchema option
loans.write.option("mergeSchema","true").format("delta").mode("append").save(DELTALAKE_SILVER_PATH)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

**Note**: With the `mergeSchema` option, we can merge these different schemas together.

In [261]:
%%sql
-- Review current loans within the `loan_by_state_delta` Delta Lake table
select addr_state, sum(`amount`) as amount from loan_by_state_delta group by addr_state order by amount desc limit 10

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

## ![Delta Lake Tiny Logo](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) Let's Travel back in Time!
Databricks Delta’s time travel capabilities simplify building data pipelines for the following use cases. 

* Audit Data Changes
* Reproduce experiments & reports
* Rollbacks

As you write into a Delta table or directory, every operation is automatically versioned.

You can query by:
1. Using a timestamp
1. Using a version number

using Python, Scala, and/or Scala syntax; for these examples we will use the SQL syntax.  

For more information, refer to [Introducing Delta Time Travel for Large Scale Data Lakes](https://databricks.com/blog/2019/02/04/introducing-delta-time-travel-for-large-scale-data-lakes.html)

### ![Delta Lake Tiny Logo](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) Review Delta Lake Table History
All the transactions for this table are stored within this table including the initial set of insertions, update, delete, merge, and inserts with schema modification

In [262]:
%%sql
DESCRIBE HISTORY loan_by_state_delta

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

### ![Delta Lake Tiny Logo](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) Time Travel via Version Number
Below are SQL syntax examples of Delta Time Travel by using a Version Number

In [200]:
version = 0
df = spark.read.format("delta").option("versionAsOf", version).load(DELTALAKE_SILVER_PATH)
df.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-----+
|addr_state|count|
+----------+-----+
|        OR|  174|
|        ND|   26|
|        DE|   37|
|        MS|   82|
|        NE|   44|
+----------+-----+
only showing top 5 rows

In [201]:
version = 9
df = spark.read.format("delta").option("versionAsOf", version).load(DELTALAKE_SILVER_PATH)
df.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-----+------------------+
|addr_state|count|            amount|
+----------+-----+------------------+
|        NM|    2|24187.511136077177|
|        NM|    0| 4325.965858470662|
|        NM|    1|10086.698942103516|
|        NM|    9|   97599.657646052|
|        NM|   15|157243.11483596396|
+----------+-----+------------------+
only showing top 5 rows